# Building a Database for Crime Reports

# Introduction

In this project, we would be working with a dataset on crimes that occurred in Boston. The dataset is available in the file `boston.csv`. The data dictionary that describes each column is as follows:
* `incident_number` : Unique ID that identifies each crime
* `offense_code` : Numeric identifier code for the crime committed
* `description` : Description of the crime committed
* `date` : Date on which the crime was commmitted
* `day_of_the_week` : The day of the week on which the crime was committed
* `lat` : Latitude coordinate of the location where the crime was committed
* `long` : Longtitude coordinate of the location where the crime was committed.

The goal of this project is to create a database named `crimes_db` with a table - `boston_crimes` - with the appropriate datatypes for storing the data from the `boston.csv` file. The table will inside a schema named `crimes`. `readonly` and `readwrite` groups with appropriate privileges will also be set up where there is one user for each of these groups. A high level overview of the intended goal is as follows:
![Image](https://dq-content.s3.amazonaws.com/250/goal.png)

# Setting Up

In [1]:
# Creating the `crimes_db` database
import psycopg2
conn = psycopg2.connect(dbname="dq", user="dq")
conn.autocommit = True
cur = conn.cursor()
cur.execute("CREATE DATABASE crime_db;")
conn.close()

In [2]:
# Connecting to the `crimes_db` database
conn = psycopg2.connect(dbname='crime_db', user='dq')
cur = conn.cursor()

In [3]:
# Creating the `crimes` schema
cur.execute("CREATE SCHEMA crimes;")
conn.commit

<function connection.commit>

# Determining Appropriate Datatypes

In [4]:
import csv
with open("boston.csv") as file:
    rows = list(csv.reader(file))
    col_headers = rows[0]
    first_row = rows[1]

In [5]:
col_headers

['incident_number',
 'offense_code',
 'description',
 'date',
 'day_of_the_week',
 'lat',
 'long']

In [6]:
first_row

['1',
 '619',
 'LARCENY ALL OTHERS',
 '2018-09-02',
 'Sunday',
 '42.35779134',
 '-71.13937053']

To help us determine to appropriate datatypes to be used for each column in the table, we would be creating an auxiliary function to create a set out of the distinct values in each column, given the name of a csv file and a column index which starts at 0, as per Python's zero-indexing.

In [7]:
def get_col_values_set(csv_filename, col_index):
    col_values_set = set()
    with open(csv_filename) as file:
        next(file) # Skipping the header row
        reader = csv.reader(file)
        for row in reader:
            col_values_set.add(row[col_index])
    return col_values_set

In [8]:
# Checking the number of different values each of the seven columns contains
col_unique_value_counts = {}
for col_idx, col_name in enumerate(col_headers):
    col_values_set = get_col_values_set("boston.csv", col_idx)
    num_unique = len(col_values_set)
    col_unique_value_counts[col_name] = num_unique

In [9]:
col_unique_value_counts

{'incident_number': 298329,
 'offense_code': 219,
 'description': 239,
 'date': 1177,
 'day_of_the_week': 7,
 'lat': 18177,
 'long': 18177}

With the above unique value counts in each column, columns with a low number of uniques values are good candidates for enumerated datatypes. In this case, the `day_of_the_week` column should be of the enumerated datatype. For textual data, we would also need to know the longest word to determine the appropriate datatype.

In [10]:
description_indx = col_headers.index("description")
description_values_set = get_col_values_set("boston.csv", description_indx)
max_len_description = len(max(description_values_set, key=len))
print(max_len_description)

58


For the description column, the maximum length is 58 characters.

# Creating the Table

From the above section on determining the appropriate datatypes for each column, the following table shows the identified suitable columns:

| Column Name | Datatype | Constraint |
| ---------   | -------- | ---------  |
| incident_number | INT | PRIMARY KEY|
| offense_code | INT | |
| description | VARCHAR(100) | |
| date | DATE | |
| day_of_the_week | day_of_the_week_enum | ENUMERATED DATATYPE|
| lat | DOUBLE PRECISION | |
| long | DOUBLE PRECISION | |

In [11]:
days_list = list(get_col_values_set("boston.csv", 4))

In [12]:
days_list

['Thursday', 'Saturday', 'Monday', 'Sunday', 'Wednesday', 'Friday', 'Tuesday']

In [13]:
# Creating the enumerated datatype for the `day_of_the_week` column
cur.execute("CREATE TYPE day_of_the_week_enum AS ENUM (%s, %s, %s, %s, %s, %s, %s);", days_list)

In [14]:
# Creating the table
cur.execute("""
CREATE TABLE crimes.boston_crimes (
incident_number INT PRIMARY KEY,
offense_code INT,
description VARCHAR(100),
date DATE,
day_of_the_week day_of_the_week_enum,
lat DOUBLE PRECISION,
long DOUBLE PRECISION)
""")
conn.commit()

# Loading the Data From CSV

With the table now ready, we can proceed to load in the data from the `boston.csv` file.

In [15]:
with open("boston.csv") as file:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", file)
    conn.commit()

# Setting Privileges

Our goal is to create two user groups: `readonly` and `readwrite`. As the names suggest, following the least privilege principle, the two user groups would only have read or read and write permissions. First, we would need to revoke all privileges inherited from the `public` group and on the `public` schema.

In [16]:
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")
conn.commit()

Next, we would need to create the user groups and the users to be added to these user groups.

In [17]:
# Creating the usergroups and granting relevant privileges
cur.execute("CREATE GROUP readonly WITH NOLOGIN;")
cur.execute("CREATE GROUP readwrite WITH NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite")
conn.commit()

In [33]:
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")
cur.execute("GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA crimes TO readwrite;")
conn.commit()

In [18]:
# Setting up one user for each group
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("GRANT readonly TO data_analyst;")
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute("GRANT readwrite TO data_scientist;")
conn.commit()
conn.close()

# Testing

In [28]:
# Setting up new conn
conn = psycopg2.connect(dbname="crime_db", user="dq")
cur = conn.cursor()

In [29]:
cur.execute("SELECT * FROM crimes.boston_crimes")
results = cur.fetchall()
for row in results[:5]:
    print(row)
conn.commit()

(1, 619, 'LARCENY ALL OTHERS', datetime.date(2018, 9, 2), 'Sunday', 42.35779134, -71.13937053)
(2, 1402, 'VANDALISM', datetime.date(2018, 8, 21), 'Tuesday', 42.30682138, -71.06030035)
(3, 3410, 'TOWED MOTOR VEHICLE', datetime.date(2018, 9, 3), 'Monday', 42.34658879, -71.07242943)
(4, 3114, 'INVESTIGATE PROPERTY', datetime.date(2018, 9, 3), 'Monday', 42.33418175, -71.07866441)
(5, 3114, 'INVESTIGATE PROPERTY', datetime.date(2018, 9, 3), 'Monday', 42.27536542, -71.09036101)


In [34]:
cur.execute("""
    SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin FROM pg_roles
    WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');
""")
for user in cur:
    print(user)

('readonly', False, False, False, False)
('readwrite', False, False, False, False)
('data_analyst', False, False, False, True)
('data_scientist', False, False, False, True)


In [35]:
cur.execute("""
    SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly', 'readwrite');
""")
for user in cur:
    print(user)
conn.close()

('readonly', 'SELECT')
('readwrite', 'INSERT')
('readwrite', 'SELECT')
('readwrite', 'UPDATE')
('readwrite', 'DELETE')
